In [65]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import os


import the notebook and use the following functions

results = search_pattern_in_data(pattern, patient_type = "all")
<!--     pattern = list or pandas series of type int -->
find_best_n_matches(results, n)


In [2]:
dep_path = 'data/condition'
nondep_path = 'data/control'
score_file_path = 'data/scores.csv'

dep_filename = sorted(os.listdir(dep_path))
nondep_filename = sorted(os.listdir(nondep_path))


In [3]:
def read_files(path):
    return pd.read_csv(path)

scores = read_files(score_file_path)

dep_files = []
nondep_files = []

for i in dep_filename:
    dep_files.append({"name":i.split('.')[0],"df":read_files(f"{dep_path}/{i}")})
    
for i in nondep_filename:
    nondep_files.append({"name":i.split('.')[0],"df":read_files(f"{nondep_path}/{i}")})

In [33]:
print((dep_files[0]["df"]["activity"]))
print(dep_files[0]["name"])

0          0
1        143
2          0
3         20
4        166
        ... 
23239      0
23240      0
23241      0
23242      0
23243    533
Name: activity, Length: 23244, dtype: int64
condition_1


In [83]:
# Pattern search
# global threshold for loss. Lower means tighter fit
global threshold
threshold = 100

def close_match(data, pattern):
    loss = 0
    min_loss = threshold
    index = -1
    n = len(pattern)
    i = 0
    while (i+n-1<len(data)):
        loss = np.sum(np.abs(np.array(data[i:i+n]) - np.array(pattern) ))
        if loss < min_loss:
            min_loss = loss
            index = i
        i = i+1
    if min_loss < threshold:
        return {"index" : index, "min_loss" : min_loss}
    else:
        return -1
    
a = [1, 2, 3, 4, 5, 6, 7]
b = [5, 5, 7]
print(close_match(a, b))
   

{'index': 4, 'min_loss': 1}


In [84]:
# patient_type = "all", "dep", "nondep" 
def search_pattern_in_data(pattern, patient_type = "all"):
    results = []
    if patient_type == "dep" or patient_type == "all":
        for patient in dep_files:
            res = close_match(patient["df"]["activity"], pattern)
            if res != -1:
                res["patient"] = patient
                results.append(res)
    elif patient_type == "non_dep" or patient_type == "all":
        for patient in nondep_files:
            res = close_match(patient["df"]["activity"], pattern)
            if res != -1:
                results.append(res)
    return results
        

In [85]:
a= np.array(dep_files[0]["df"]["activity"])
pattern = [  143,   0,  20, 166, 160,  17, 646, 978, 306, 200]

In [86]:
print(close_match(a, pattern))
   

{'index': 1, 'min_loss': 77}


In [87]:
print(np.sum(np.abs(a[9004:9004+len(pattern)]-np.array(pattern))))
print(a[9004:9004+len(pattern)])
print(np.array(pattern))
print((a[9004:9004+len(pattern)]-np.array(pattern)))

2362
[286 143 148 349 219 154  82 361 361 533]
[143   0  20 166 160  17 646 978 306 200]
[ 143  143  128  183   59  137 -564 -617   55  333]


In [88]:
a[0:11]

array([  0, 143,   0,  20, 166, 160,  17, 646, 978, 306, 277], dtype=int64)

In [89]:
results = search_pattern_in_data(pattern)
best_loss = threshold
ind = -1
for i,r in enumerate(results):
    if results[i]["min_loss"]<best_loss:
        best_loss = results[i]["min_loss"]
        ind = i
print(results[ind])
    

{'index': 1, 'min_loss': 77, 'patient': {'name': 'condition_1', 'df':                  timestamp        date  activity
0      2003-05-07 12:00:00  2003-05-07         0
1      2003-05-07 12:01:00  2003-05-07       143
2      2003-05-07 12:02:00  2003-05-07         0
3      2003-05-07 12:03:00  2003-05-07        20
4      2003-05-07 12:04:00  2003-05-07       166
...                    ...         ...       ...
23239  2003-05-23 15:19:00  2003-05-23         0
23240  2003-05-23 15:20:00  2003-05-23         0
23241  2003-05-23 15:21:00  2003-05-23         0
23242  2003-05-23 15:22:00  2003-05-23         0
23243  2003-05-23 15:23:00  2003-05-23       533

[23244 rows x 3 columns]}}


In [90]:
def find_best_n_matches(results, n):
    newlist = sorted(results, key=lambda d: d['min_loss'])
    
    return newlist[:n]

find_best_n_matches(results,3)

[{'index': 1,
  'min_loss': 77,
  'patient': {'name': 'condition_1',
   'df':                  timestamp        date  activity
   0      2003-05-07 12:00:00  2003-05-07         0
   1      2003-05-07 12:01:00  2003-05-07       143
   2      2003-05-07 12:02:00  2003-05-07         0
   3      2003-05-07 12:03:00  2003-05-07        20
   4      2003-05-07 12:04:00  2003-05-07       166
   ...                    ...         ...       ...
   23239  2003-05-23 15:19:00  2003-05-23         0
   23240  2003-05-23 15:20:00  2003-05-23         0
   23241  2003-05-23 15:21:00  2003-05-23         0
   23242  2003-05-23 15:22:00  2003-05-23         0
   23243  2003-05-23 15:23:00  2003-05-23       533
   
   [23244 rows x 3 columns]}}]

In [69]:
print((dep_files[0]["df"]["activity"])[0:10])
print((dep_files[10]["df"]["activity"])[6590:6600])
print((dep_files[11]["df"]["activity"])[14284:14294])

0      0
1    143
2      0
3     20
4    166
5    160
6     17
7    646
8    978
9    306
Name: activity, dtype: int64
6590     0
6591     0
6592     0
6593     0
6594     0
6595     0
6596     0
6597     0
6598    54
6599     0
Name: activity, dtype: int64
14284    624
14285    116
14286    306
14287     19
14288     50
14289    234
14290    125
14291    360
14292    296
14293    160
Name: activity, dtype: int64
